## Install and Import Dependicies

In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ---------------------------------------- 2.1/2.1 GB 932.0 kB/s eta 0:00:00
     ---------------------------------------- 4.7/4.7 MB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 3.2 MB/s eta 0:00:00


In [2]:
! pip install transformers requests beautifulsoup4 

  Using cached transformers-4.22.2-py3-none-any.whl (4.9 MB)
     -------------------------------------- 163.5/163.5 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 3.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\suley\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\suley\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

## Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode("I hated this, absolutely the worst", return_tensors="pt")

In [6]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [8]:
result = model(tokens)

In [9]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [13]:
int(torch.argmax(result.logits) + 1)

1

In [17]:
tokens = tokenizer.encode("This is amazing, I loved it. GREAT!", return_tensors="pt")

In [18]:
result = model(tokens)

In [19]:
int(torch.argmax(result.logits) + 1)

5

In [20]:
tokens = tokenizer.encode("It was okay!", return_tensors="pt")

In [21]:
result = model(tokens)

In [22]:
int(torch.argmax(result.logits) + 1)

3

## Collect Reviews

In [44]:
r = requests.get("https://www.yelp.com/biz/nusr-et-steakhouse-istanbul-9")
soup = BeautifulSoup(r.text, "html.parser")
regex = re.compile(".*comment.*")
results = soup.find_all("p", {"class":regex})
reviews = [result.text for result in results]

In [45]:
results[0].text

"O my god it's the best. I hated the one in dubai, it was too expensive and the service was awful, this covered my last experience completely. The service was amazing in Istanbul and the food quality was above expectation. I only wish they improve their dubai quality and service to this standard."

## Load Reviews into DataFrame and Score

In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.DataFrame(np.array(reviews), columns = ["review"])
df.head()

,review
0,O my god it's the best. I hated the one in dub...
1,Writing this review for the service. The food ...
2,It's extremely touristy and super overrated. T...
3,"It's good. Fantastic. However, I think it is o..."
4,Amazing steak! service also good quality but p...


In [48]:
df.loc[0, "review"]

"O my god it's the best. I hated the one in dubai, it was too expensive and the service was awful, this covered my last experience completely. The service was amazing in Istanbul and the food quality was above expectation. I only wish they improve their dubai quality and service to this standard."

In [49]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits) + 1)

In [50]:
sentiment_score(df.loc[0, "review"])

5

In [51]:
df["sentiment"] = df["review"].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,O my god it's the best. I hated the one in dub...,5
1,Writing this review for the service. The food ...,2
2,It's extremely touristy and super overrated. T...,2
3,"It's good. Fantastic. However, I think it is o...",4
4,Amazing steak! service also good quality but p...,5
5,Great food overpriced and little overrated We ...,3
6,I may be one of very few who have this opinion...,3
7,THE BEST STEAK HOUSE SERVICE !! If you're in T...,5
8,This is the original restaurant of the Instagr...,4
9,Not sure why this place doesn't have a better ...,3


In [52]:
df.loc[2, "review"]

'It\'s extremely touristy and super overrated. The lines are long people dress up and it\'s like the equivalent of going to outback steakhouse in America. I understand the gimmick with the whole salt Bae thing.. Bey..?It worked. The place is always crowded with tourists. They make great fries and a pretty good burger but you\'ll find the same for half the price at any burger place place within the 10 mile radius of the restaurant.You cannot order lamb unless you\'re there with family because the portions are huge. Five people. It\'s sort of a " thing to do" like visit \xa0katz deli when you\'re in New York.*Do not order the steak salad because it is sliced deli meat on top of arugula..For $30 USDThat\'s almost 250 Turkish lira.For a few slices of processed meat'